In [4]:
import pandas as pd
import numpy as np

In [5]:
# Load your dataset
df = pd.read_excel("../data/Diabetes_ALL.xlsx", sheet_name="Diab_RAW")

In [6]:
# Clean column names
df.columns = df.columns.str.strip().str.replace(' ', '_')

In [7]:
df.head()

,Age,Gender,Gestational_Diabetes,Marital_Status,Family_Income,Education,Occupation,Residence,District,Family_History,High_Blood_Pressure,Physically_Active,Height,Weight_Status
0,50-59 Years,Male,No,Married,">50,000BDT",Higher Secondary,Business,Rural,Dhaka,Yes,Yes,Yes,5’5’’ (165cm),189-226 lbs. (86-102kg)
1,Less than 40 Years,Male,NaN,Unmarried,"20,000-50,000 BDT",Undergraduate,Student,Urban,Noakhali,Yes,No,Yes,5’10’’ (178cm),189-226 lbs. (86-102kg)
2,Less than 40 Years,Female,No,Unmarried,"20,000-50,000 BDT",Undergraduate,Unemployed,Urban,Comilla,Yes,No,Yes,4’11’’ (150cm),<189 lbs. (86kg)
3,Less than 40 Years,Female,No,Unmarried,"20,000-50,000 BDT",Undergraduate,Lecturer,Rural,Comilla,Yes,No,Yes,5’0’’ (152cm),<189 lbs. (86kg)
4,Less than 40 Years,Male,NaN,Unmarried,">50,000BDT",Undergraduate,Student,Semi-urban,Pabna,No,No,No,5’8’’ (173cm),189-226 lbs. (86-102kg)


In [8]:
df.shape

(5026, 14)

In [5]:
# Convert height and weight from text to numeric
# Extract height in cm
def extract_height_cm(value):
    if pd.isna(value):
        return np.nan
    if '(' in value:
        try:
            return float(value.split('(')[-1].replace('cm)', '').strip())
        except:
            return np.nan
    return np.nan

# Extract average weight in kg
def extract_weight_kg(value):
    if pd.isna(value):
        return np.nan
    try:
        weight_range = value.split('(')[-1].replace('kg)', '')
        parts = weight_range.split('-')
        if len(parts) == 2:
            return (float(parts[0]) + float(parts[1])) / 2
        else:
            return float(parts[0].replace('<', ''))
    except:
        return np.nan

df['Height_cm'] = df['Height'].apply(extract_height_cm)
df['Weight_kg'] = df['Weight_Status'].apply(extract_weight_kg)

In [6]:
# Convert Yes/No to 1/0
binary_cols = ['Gestational_Diabetes', 'Family_History', 'High_Blood_Pressure', 'Physically_Active']
for col in binary_cols:
    df[col] = df[col].map({'Yes': 1, 'No': 0})

In [7]:
# Define the scoring function
def assign_risk_score(row):
    points = 0

    # 1. Age
    if row['Age'] == '40-49 Years':
        points += 1
    elif row['Age'] == '50-59 Years':
        points += 2
    elif row['Age'] == '60+ Years':
        points += 3

    # 2. Gender
    if row['Gender'] == 'Male':
        points += 1

    # 3. Gestational Diabetes
    if row['Gender'] == 'Female' and row['Gestational_Diabetes'] == 1:
        points += 1

    # 4. Family History
    if row['Family_History'] == 1:
        points += 1

    # 5. High Blood Pressure
    if row['High_Blood_Pressure'] == 1:
        points += 1

    # 6. Physically Active
    if row['Physically_Active'] == 0:
        points += 1

    # 7. Weight Category (based on height and weight)
    height_weight_chart = {
        147: (124, 147, 198), 150: (128, 152, 204), 152: (132, 157, 211), 155: (136, 163, 218),
        157: (141, 168, 225), 160: (145, 173, 232), 165: (150, 179, 240), 168: (155, 185, 247),
        170: (159, 190, 254), 173: (164, 196, 261), 175: (169, 202, 269), 178: (174, 208, 277),
        180: (179, 214, 285), 183: (184, 220, 294), 185: (189, 226, 302), 188: (194, 232, 311),
        191: (200, 239, 319), 193: (205, 245, 327)
    }

    height = row['Height_cm']
    weight = row['Weight_kg']
    if pd.notnull(height) and pd.notnull(weight):
        rounded_height = round(height)
        if rounded_height in height_weight_chart:
            low, mid, _ = height_weight_chart[rounded_height]
            if weight < low:
                points += 0
            elif low <= weight <= mid:
                points += 1
            else:
                points += 2

    return points

In [8]:
df['Risk_Score'] = df.apply(assign_risk_score, axis=1)
df['Risk_Category'] = df['Risk_Score'].apply(lambda x: 'High Risk (≥5)' if x >= 5 else 'Low Risk (<5)')

In [9]:
df.head()

,Age,Gender,Gestational_Diabetes,Marital_Status,Family_Income,Education,Occupation,Residence,District,Family_History,High_Blood_Pressure,Physically_Active,Height,Weight_Status,Height_cm,Weight_kg,Risk_Score,Risk_Category
0,50-59 Years,Male,0.0,Married,">50,000BDT",Higher Secondary,Business,Rural,Dhaka,1.0,1.0,1.0,5’5’’ (165cm),189-226 lbs. (86-102kg),165.0,94.0,5,High Risk (≥5)
1,Less than 40 Years,Male,NaN,Unmarried,"20,000-50,000 BDT",Undergraduate,Student,Urban,Noakhali,1.0,0.0,1.0,5’10’’ (178cm),189-226 lbs. (86-102kg),178.0,94.0,2,Low Risk (<5)
2,Less than 40 Years,Female,0.0,Unmarried,"20,000-50,000 BDT",Undergraduate,Unemployed,Urban,Comilla,1.0,0.0,1.0,4’11’’ (150cm),<189 lbs. (86kg),150.0,86.0,1,Low Risk (<5)
3,Less than 40 Years,Female,0.0,Unmarried,"20,000-50,000 BDT",Undergraduate,Lecturer,Rural,Comilla,1.0,0.0,1.0,5’0’’ (152cm),<189 lbs. (86kg),152.0,86.0,1,Low Risk (<5)
4,Less than 40 Years,Male,NaN,Unmarried,">50,000BDT",Undergraduate,Student,Semi-urban,Pabna,0.0,0.0,0.0,5’8’’ (173cm),189-226 lbs. (86-102kg),173.0,94.0,2,Low Risk (<5)


In [10]:
# Export to Excel
df.to_excel("../data/Preprocessed_Diabetes_Data.xlsx", index=False)